## Import libraries

In [3]:
!pip install scikit-learn --quiet


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score, f1_score

## Utils

## Read Data

In [9]:
df_ratings.head()

,Unnamed: 0,user_id,item_id,rating
0,0,196,242,3
1,1,186,302,3
2,2,22,377,1
3,3,244,51,2
4,4,166,346,1


In [10]:
df_ratings = pd.read_csv("/home/mariam/PycharmProjects/recommendation-systems/data/preprocessed/ratings.csv")

user_item_matrix = df_ratings.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

## Model

In [11]:
# Specify the number of folds for cross-validation
n_splits = 5

# Set a random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Initialize the k-NN model
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')

In [12]:
def get_k_nearest_neighbors(user_index, k=5):
    distances, indices = knn_model.kneighbors([user_item_matrix.iloc[user_index]], n_neighbors=k+1)
    
    # Exclude the first item, which is the user itself
    distances = distances.flatten()[1:]
    indices = indices.flatten()[1:]
    
    return list(zip(indices, distances))

In [39]:
# Initialize KFold with manual seeding
kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
k_values = [5, 10, 20]
# Lists to store evaluation results for each fold
precision_scores = {k: [] for k in k_values}
recall_scores = {k: [] for k in k_values}
f1_scores = {k: [] for k in k_values}

# Perform k-fold cross-validation
for train_index, test_index in kf.split(user_item_matrix):
    # Split the data into training and testing sets
    train_data, test_data = user_item_matrix.iloc[train_index], user_item_matrix.iloc[test_index]
    
    # Fit the k-NN model on the training data
    knn_model.fit(train_data)
    
    # Evaluate on the test data
    for user_index in test_index:
        # Get the ground truth (items liked by the user in the test set)
        ground_truth= deepcopy(user_item_matrix.iloc[40])
        ground_truth = ground_truth[ground_truth > 3]
        ground_truth = set(ground_truth.index.values)
        
        
        # Get k-nearest neighbors
        nearest_neighbors = get_k_nearest_neighbors(user_index, max(k_values))
        
        for k in k_values:
            # Extract the top k recommended items from neighbors
            top_k_recommendations = set(item for item, _ in nearest_neighbors[:k])
            
            # Calculate precision, recall, and F1 score at k
            precision = len(ground_truth.intersection(top_k_recommendations)) / k
            recall = len(ground_truth.intersection(top_k_recommendations)) / len(ground_truth)
            
            # Avoid division by zero
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            precision_scores[k].append(precision)
            recall_scores[k].append(recall)
            f1_scores[k].append(f1)

# Calculate average precision, recall, and F1 score across all folds for each k
avg_precision = {k: np.mean(scores) for k, scores in precision_scores.items()}
avg_recall = {k: np.mean(scores) for k, scores in recall_scores.items()}
avg_f1 = {k: np.mean(scores) for k, scores in f1_scores.items()}

# Display results
print("Average Precision at k:")
for k, score in avg_precision.items():
    print(f"At k={k}: {score:.4f}")

print("\nAverage Recall at k:")
for k, score in avg_recall.items():
    print(f"At k={k}: {score:.4f}")

print("\nAverage F1 Score at k:")
for k, score in avg_f1.items():
    print(f"At k={k}: {score:.4f}")

Average Precision at k:
At k=5: 0.0456
At k=10: 0.0453
At k=20: 0.0435

Average Recall at k:
At k=5: 0.0063
At k=10: 0.0126
At k=20: 0.0242

Average F1 Score at k:
At k=5: 0.0111
At k=10: 0.0197
At k=20: 0.0311


In [37]:
from copy import deepcopy

ground_truth= deepcopy(user_item_matrix.iloc[40])

ground_truth = ground_truth[ground_truth > 3]
ground_truth = set(ground_truth.index.values)
# x.sort_values(inplace=True, ascending=False)
# indices = x.index.astype(int)
# set(indices.values)

In [38]:
ground_truth

{1,
 28,
 50,
 56,
 69,
 96,
 98,
 100,
 135,
 152,
 153,
 156,
 168,
 170,
 173,
 174,
 175,
 180,
 181,
 188,
 191,
 195,
 205,
 209,
 238,
 286,
 318,
 357,
 414,
 430,
 474,
 486,
 514,
 516,
 751,
 969}

In [19]:
get_k_nearest_neighbors(40,10)

[(587, 0.5533295106986258),
 (77, 0.5629883607656635),
 (413, 0.5738081498762717),
 (746, 0.5878436523774835),
 (558, 0.594197378128124),
 (429, 0.6008199031694375),
 (328, 0.6033888739840246),
 (513, 0.6041479409599898),
 (692, 0.6063239886714646),
 (311, 0.6120980294668037)]

In [ ]:
# da kda ana m3aya lel user da k ground truth X movies and their ratings
# w m3aya recommended 10, fa hashof hal homa included fe el rated 5 and 4 wla laa?